# Udemy講座「LangChainによる大規模言語モデル（LLM）アプリケーション開発入門」セクション「（アップデート）OpenAI の Chat API の Function calling 機能について」のソースコード

## 1. Function calling の基本

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

In [ ]:
!pip install --quiet langchain==0.0.229 openai==0.27.8

以下はOpenAIの公式のサンプルコードをもとに一部変更（printの追加など）したソースコードです。

参考：https://platform.openai.com/docs/guides/gpt/function-calling

In [ ]:
import openai
import json


# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)


def run_conversation():
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": "What's the weather like in Boston?"}]
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    print("=== first response ===")
    print(response)
    response_message = response["choices"][0]["message"]

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        fuction_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = fuction_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )
        print("=== python function output ===")
        print(function_response)

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        print("=== second request messages ===")
        print(json.dumps(messages, indent=2))
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        print("=== second response ===")
        print(second_response)


run_conversation()

## 2. Function calling を使った LangChain の「OpenAI Functions Agent」

In [ ]:
import json

import openai
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.chat_models import ChatOpenAI

def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

tools = [
    Tool(
        name="get_current_weather",
        func=get_current_weather,
        description="Get the current weather in a given location"
    )
]

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = initialize_agent(tools, chat, agent=AgentType.OPENAI_FUNCTIONS)

result = agent.run("What's the weather like in Boston?")
print(result)


In [ ]:
import openai
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
tools = load_tools(["terminal", "llm-math"], llm=chat)
agent = initialize_agent(tools, chat, agent=AgentType.OPENAI_FUNCTIONS)

result = agent.run("Show files in ./sample_data directory.")
print(result)

## 3. Function calling を応用した Extraction と Tagging

以下はLangChainの公式ドキュメントのこちらのページを参考にしたサンプルコードです。

https://python.langchain.com/docs/modules/chains/additional/extraction

※ LangChainの公式ドキュメントは高い頻度でリンク切れになります。もしリンク切れになっている場合は、LangChainのドキュメントで「Extraction」と検索してみてください。

In [ ]:
import json

import openai
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain, create_extraction_chain_pydantic
from langchain.prompts import ChatPromptTemplate

openai.log = "debug"
langchain.verbose = True

schema = {
    "properties": {
        "person_name": {"type": "string"},
        "person_height": {"type": "integer"},
        "person_hair_color": {"type": "string"},
        "dog_name": {"type": "string"},
        "dog_breed": {"type": "string"},
    },
    "required": ["person_name", "person_height"],
}
inp = """
Alex is 5 feet tall. Claudia is 1 feet taller Alex and jumps higher than him. Claudia is a brunette and Alex is blonde.
Alex's dog Frosty is a labrador and likes to play hide and seek.
"""

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = create_extraction_chain(schema, chat)

result = chain.run(inp)
print(f"""=== result ===
{json.dumps(result, indent=2)}
===""")

以下はLangChainの公式ドキュメントのこちらのページを参考にしたサンプルコードです。

https://python.langchain.com/docs/modules/chains/additional/tagging

※ LangChainの公式ドキュメントは高い頻度でリンク切れになります。もしリンク切れになっている場合は、LangChainのドキュメントで「Tagging」と検索してみてください。

In [ ]:
import openai
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic
from langchain.prompts import ChatPromptTemplate

openai.log = "debug"
langchain.verbose = True

schema = {
    "properties": {
        "sentiment": {"type": "string"},
        "aggressiveness": {"type": "integer"},
        "language": {"type": "string"},
    }
}

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = create_tagging_chain(schema, chat)

inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"

result = chain.run(inp)
print(f"""=== result ===
{json.dumps(result, indent=2)}
===""")

## 4. 1 度に複数の関数を実行できる LangChain の「OpenAI Multi Functions Agent」

以下はLangChainの公式ドキュメントのこちらのページを参考にしたサンプルコードです。

https://python.langchain.com/docs/modules/agents/agent_types/openai_multi_functions_agent

※ LangChainの公式ドキュメントは高い頻度でリンク切れになります。もしリンク切れになっている場合は、LangChainのドキュメントで「OpenAI Multi Functions Agent」などと検索してみてください。

In [ ]:
# duckduckgo-searchのバージョン3.8.3は動作しなくなったため、バージョン4.1.0をインストールします

!pip install --quiet duckduckgo-search==4.1.0

In [ ]:
import langchain
import openai
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.chat_models import ChatOpenAI

langchain.debug = True
langchain.verbose = False
openai.log = "info"

tools = load_tools(["ddg-search"])
chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = initialize_agent(tools, chat, agent=AgentType.OPENAI_MULTI_FUNCTIONS)

result = agent.run(
    "What is the weather in LA and SF?"
)
print(f"""=== result ===
{result}
===""")